In [ ]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import sys

def get_img_agnostic_mask(img, pose_data):

    r = 20
    agnostic = Image.new('RGB', (img.size[0], img.size[1]))
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
    r = int(length_a / 16) + 1

    # mask arms
    #agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'white', width=r*10)
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'white', 'white')
    for i in [10,11,13,14]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'white', width=r*10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'white', 'white')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'white', 'white')
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 11]], 'white', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [12, 14]], 'white', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'white', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [9, 12, 14, 11]], 'white', 'white')

    return agnostic
    
human_image_path = "../sample_images/human_image"
data_path = "../sample_images"
os.makedirs(osp.join(data_path, "cloth_agnostic_mask"), exist_ok=True)


for im_name in os.listdir(human_image_path):        
    # load pose image
    pose_name = im_name.replace('.jpg', '_keypoints.json')

    if os.path.isfile(osp.join(data_path, 'human_keypoints_posed', pose_name)):

        with open(osp.join(data_path, 'human_keypoints_posed', pose_name), 'r') as f:
            pose_label = json.load(f)
            pose_data = pose_label['people'][0]['pose_keypoints_2d']
            pose_data = np.array(pose_data)
            pose_data = pose_data.reshape((-1, 3))
            pose_data = pose_data[:, :2]
        
        # load parsing image
        im = Image.open(osp.join(human_image_path, im_name))

        agnostic = get_img_agnostic_mask(im, pose_data)
        
        agnostic.save(osp.join(data_path, "cloth_agnostic_mask", im_name.replace('.jpg', '.png')))